In [ ]:
project_path = "/home/jupyter"
import os
import sys

sys.path.append(project_path)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import plotly.express as px
from datetime import datetime
from google.cloud import bigquery

from fintrans_toolbox.src import bq_utils as bq
from fintrans_toolbox.src import table_utils as t

import ft_digital_trade.src.utils.read_data as read_utils
import ft_digital_trade.src.utils.clean_utils as clean_utils
import ft_digital_trade.src.utils.calculation_utils as calc_utils
import ft_digital_trade.src.utils.plot_utils as plot_utils

client = bigquery.Client()

In [ ]:
#Looking at all transactions by UK cardholders, online, on all MCCs related to goods from Rest of ASIAPAC
UK_De_Min = """SELECT time_period_value, spend, transactions, mcc, cardholder_location
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel`
WHERE time_period = 'Month'
and merchant_channel = 'Online'
and cardholder_origin = 'UNITED KINGDOM'
and mcc in ('ANTIQUE REPRODUCTION STORES',
'ANTIQUE SHOPS', 
        'ARTIST/CRAFT SHOPS',
        'ART DEALERS & GALLERIES',
        'AUTOMATED FUEL DISPENSERS',
        'AUTOMOTIVE PARTS STORES', 
        'AUTOMOTIVE TIRE STORES', 
        'BAKERIES', 
        'BICYCLE SHOPS/SALE/SERVICE',
        'BOAT DEALERS',
        'BOOKS/PERIODICALS/NEWSPAPERS',
        'BOOK STORES', 
        'CAMERA & PHOTO SUPPLY STORES',
        'CAMPER TRAILER DEALER',
        'CANDY/NUT/CONFECTION STORES',
        'CAR & TRUCK DEALERS/NEW/USED', 
        'CAR & TRUCK DEALERS/USED ONLY', 
        'CATALOG MERCHANT',
        'CHILDREN/INFANTS WEAR STORES',
        'CLOTHING/RENT/COSTUME/UNIFO',
        'COMBINATION CATALOG & RETAIL',
        'COMPUTERS/PERIPHERALS/SOFTWARE',
        'COSMETIC STORES', 
        'DAIRY PRODUCT STORES',
        'DEPARTMENT STORES', 
        'DIRECT SELL/DOOR-TO-DOOR',
        'DISCOUNT STORES',
        'DRAPERY & UPHOLSTERY STORES',
        'DUTY FREE STORES',
        'ELEC RAZOR STORES/SALE/SERV',
        'ELECTRONICS STORES', 
        'FABRIC STORES',
        'FAMILY CLOTHING STORES',
        'FIREPLACES & ACCESSORIES',
        'FLOOR COVERING STORES', 
        'FLORIST SUPPLIES/NURSERY STOCK',
        'FLORISTS',
        'FREEZER/MEAT LOCKERS',
        'FUEL DEALERS',
        'FURNITURE/EQUIP STORES',
        'FURRIERS AND FUR SHOPS',
        'GLASS/PAINT/WALLPAPER STORES', 
        'GIFT, CARD, NOVELTY STORES', 
        'GLASSWARE/CRYSTAL STORES',
        'GROCERY STORES/SUPERMARKETS',
        'HARDWARE STORES',
        'HOBBY, TOY & GAME STORES',
        'HOME SUPPLY WAREHOUSE STORES',
        'HOUSEHOLD APPLIANCE STORES', 
        'JEWELRY STORES', 
        'LUGGAGE/LEATHER STORES',
        'LUMBER/BUILD. SUPPLY STORES', 
        'MEN/BOYS CLOTHING/ACC STORES',
        'MENS/WOMENS CLOTHING STORES',
        'MISC APPAREL/ACCESS STORES',
        'MISC AUTO DEALERS - DEFAULT',
        'MISC FOOD STORES - DEFAULT',
        'MISC GENERAL MERCHANDISE', 
        'MISC HOME FURNISHING SPECIALTY', 
        'MISC SPECIALTY RETAIL', 
        'MOBILE HOME DEALERS', 
        'MOTOR HOME DEALERS',
        'MOTOR VEHICLE SUPPLY/NEW PARTS', 
        'MOTORCYCLE DEALERS', 
        'MUSIC STORES/PIANOS', 
        'NEWS DEALERS/NEWSSTANDS',
        'NURSURIES, LAWN/GARDEN SUPPLY',
        'OFFICE/PHOTO EQUIPMENT',
        'ONLINE MARKETPLACES',
        'OTHER DIRECT MARKETERS',
        'OUTBOUND TELEMARKETING MERCHNT',
        'PAINT, VARNISHES & SUPPLIES',
        'PET STORES/FOOD & SUPPLY',
        'PETROLEUM/PETROLEUM PRODUCTS',
        'PLUMBING/HEATING EQUIPMENT',
        'POSTAGE STAMPS',
        'PRECIOUS STONES/METALS/JEWELRY',
        'RECORD STORES',
        'RELIGIOUS GOODS STORES',
        'ROOFING/SIDING/SHEET METAL'
        'SERVICE STATIONS',
        'SHOE STORES',
        'SNOWMOBILE DEALERS',
        'SPORTING GOODS STORES',
        'SPORTS/RIDING APPAREL STORES',
        'STAMP & COIN STORES',
        'STATIONERY STORES',
        'STATIONERY/OFFICE SUPPLIES',
        'SWIMMING POOLS/SALES/SERV',
        'TENT AND AWNING SHOPS',
        'TELECOMMUNICATION EQUIPMENT',
        'UNIFORMS & COMMERCIAL CLOTHING',
        'USED MERCHANDISE STORES',
        'WIG AND TOUPEE STORES',
        'WOMENS READY TO WEAR STORES',
        'WRECKING SALVAGE YARDS',
'VIDEO AMUSEMENT GAME SUPPLY')
and destination_country = 'REST OF  ASIAPAC'
and cardholder_origin_country = 'POSTAL_AREA'
GROUP BY time_period_value, mcc, spend, transactions, cardholder_location
ORDER BY time_period_value, mcc, cardholder_location
"""
df_De_Min = bq.read_bq_table_sql(client, UK_De_Min)
p_area_sum = df_De_Min.groupby('time_period_value')['spend'].sum().reset_index()
df_De_Min

In [ ]:
#Z score table - assuming a normal distribution around the mean
z_table = {
    "Z": [-4,-3.9,-3.8,-3.7,-3.6,-3.5,-3.4,-3.3,-3.2,-3.1,-3,-2.9,-2.8,-2.7,-2.6,-2.5,-2.4,-2.3,-2.2,-2.1,-2,-1.9,-1.8,-1.7,-1.6,-1.5,1.4,-1.3,-1.2,-1.1,-1,-0.9,
          -0.8,-0.7,-0.6,-0.5,-0.4,-0.3,-0.2,-0.1,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2,2.1,2.2,2.3,2.4,2.5,2.6,2.7,2.8,
          2.9,3,3.1,3.2,3.3,3.4,3.5,3.6,3.7,3.8,3.9,4],
    "De Min %": [0.00003,0.00005,0.00007,0.00011,0.00016,0.00023,0.00034,0.00048,0.00069,0.00097,0.00135,0.00187,0.00256,0.00347,0.00466,0.00621,0.0082,0.01072,
                0.0139,0.01786,0.02275,0.02872,0.03593,0.04457,0.0548,0.06681,0.08076,0.0968,0.11507,0.13567,0.15866,0.18406,0.21186,0.24196,0.27425,0.30854,
                0.34458,0.38209,0.42074,0.46017,0.5,0.53983,0.57926,0.61791,0.65542,0.69146,0.72575,0.75804,0.78814,0.81594,0.84134,0.86433,0.88493,0.9032,
                0.91924,0.93319,0.9452,0.95543,0.96407,0.97128,0.97725,0.98214,0.9861,0.98928,0.9918,0.99379,0.99534,0.99653,0.99744,0.99813,0.99865,0.99903,
                0.99931,0.99952,0.99966,0.99977,0.99984,0.99989,0.99993,0.99995,0.99997]
}
df_z_table = pd.DataFrame(z_table)
df_z_table

In [ ]:
#Calculating the Average transaction value for each MCC in each time period
df_De_Min['Avg Transaction'] = (df_De_Min['spend'] / df_De_Min['transactions'])
df_De_Min

In [ ]:
#You can generate a rough standard deviation using what’s called an interquartile range,
#which is the difference between the 25th percentile threshold and the 75th percentile threshold, where the central 50% of outcomes lie. 
#When sample sizes are large and the distribution of the outcome is similar to the normal distribution,
#the width of the interquartile range will be approximately 1.35 standard deviations.

#Starting by calculating the upper and lower range using the assumption 50% of transactions will be +/- 25% of the avg transaction value
df_De_Min['Upper Range'] = df_De_Min['Avg Transaction']*1.25
df_De_Min['Lower Range'] = df_De_Min['Avg Transaction']*0.75

#Setting our constants
interquartile_range = 1.35
de_minimis_trade_threshold = 135

#Calculating standard deviation and the z value for each postcode/mcc/month combination
df_De_Min['std_dev_value'] = (df_De_Min['Upper Range'] - df_De_Min['Lower Range']) / interquartile_range
df_De_Min['Z Value'] = (de_minimis_trade_threshold-df_De_Min['Avg Transaction'])/df_De_Min['std_dev_value']
df_De_Min['Z Value'] = df_De_Min['Z Value'].round(1)
df_De_Min

In [ ]:
#Using the z-value table to determine which percentage of the spend is de minimis
def z_value_lookup(row):
    if row['Z Value'] > 4:
        val = 1
    elif row['Z Value'] < -4:
        val = 0
    else:
        z_score = row['Z Value']
        val = df_z_table.loc[df_z_table['Z'] == z_score, 'De Min %'].values
        if len(val) > 0:
            val = val[0]
        else:
            val = None
    return val

#Apply the lookout to add the estimated % of spend that is de min to each row
df_De_Min['% De Min'] = df_De_Min.apply(z_value_lookup, axis=1)
df_De_Min

In [ ]:
#Calculating our estimate de minimis trade value based on calculated % that is de min
df_De_Min['Estimated De Minimis Trade Spend'] = df_De_Min['spend']*df_De_Min['% De Min']
df_De_Min

In [ ]:
df_De_Min.to_csv('de_minimis_trade.csv')

In [ ]:
#Cleaning up the table
df_De_Min = df_De_Min.drop('Upper Range', axis=1)
df_De_Min = df_De_Min.drop('Lower Range', axis=1)
df_De_Min = df_De_Min.drop('std_dev_value', axis=1)
df_De_Min = df_De_Min.drop('Z Value', axis=1)
df_De_Min = df_De_Min.drop('% De Min', axis=1)

neworder = ['time_period_value','spend','Estimated De Minimis Trade Spend','transactions','Avg Transaction', 'mcc','cardholder_location']
df_De_Min=df_De_Min.reindex(columns=neworder)
df_De_Min

In [ ]:
df_De_Min.to_csv('clean_de_minimis_trade2.csv')

In [ ]:
# Adding on yearly values to table
df_De_Min['Year'] = df_De_Min['time_period_value'].str[:4]
num_years = pd.to_numeric(df_De_Min['Year'], errors='coerce')
df_De_Min['Year'] = num_years
df_De_Min

In [ ]:
# Line chart using plotly express
pfig = px.line(
        df_De_Min,
        x="time_period_value",
        y="Estimated De Minimis Trade Spend",
        color="destination_country")
pfig

In [ ]:
pfig2 = px.line(
        df_De_Min,
        x="time_period_value",
        y="spend",
        color="destination_country")
pfig2

In [ ]:
#df_De_Min.to_csv('de_minimis_trade.csv')

In [ ]:
# Visa marketshare values for spend from UK Finance and transactions from Global Data report using 50 and 45 as fillers for 2024 for now
data = {
    'Year': [2019, 2020, 2021, 2022, 2023, 2024],
    'Spend Marketshare': [83.76315246544253, 84.70314576871952, 80.37968292436635, 67.12391403749429, 59.3082835183603769, 55],
    'Transactions Marketshare': [57.47489830420878, 61.862124334736365, 59.25612008522669, 51.8065116354824, 46.67942317540564, 45]
}
visa_marketshare = pd.DataFrame(data)
visa_marketshare

In [ ]:
# Adding the marketshare values to data table
df_De_Min_mkt = df_De_Min
df_De_Min_mkt = pd.merge(df_De_Min, visa_marketshare, on='Year', how='outer')
df_De_Min_mkt

In [ ]:
# Calculating total number of transactions and total spend (based on above marketshare values)
total_spend_de_min = df_De_Min_mkt
total_spend_de_min['Total Spend'] = total_spend_de_min['spend'] * (100 / total_spend_de_min['Spend Marketshare'])
total_spend_de_min['Total Transactions'] = total_spend_de_min['transactions'] * (100 / total_spend_de_min['Transactions Marketshare'])
#total_spend_de_min = total_spend_de_min.drop('spend', axis=1)
#total_spend_de_min = total_spend_de_min.drop('transactions', axis=1)
total_spend_de_min

In [ ]:
# Cleaning up the table
total_spend_de_min = total_spend_de_min.drop('spend', axis=1)
total_spend_de_min = total_spend_de_min.drop('transactions', axis=1)
total_spend_de_min = total_spend_de_min.drop('Year', axis=1)
total_spend_de_min = total_spend_de_min.drop('Spend Marketshare', axis=1)
total_spend_de_min = total_spend_de_min.drop('Transactions Marketshare', axis=1)
total_spend_de_min

In [ ]:
# Re-calculating average transaction value based on scaled up spend+transactions values
total_spend_de_min['Total Mkt Avg Transaction'] = (total_spend_de_min['Total Spend'] / total_spend_de_min['Total Transactions'])
total_spend_de_min['Visa Avg Transaction'] = total_spend_de_min['Avg Transaction']
total_spend_de_min = total_spend_de_min.drop('Avg Transaction', axis=1)
total_spend_de_min

In [ ]:
# Filter out rows where the Avg Transaction is greater than 135 (just de minimis trade)
total_spend_de_min = total_spend_de_min[total_spend_de_min['Total Mkt Avg Transaction'] < 135]
total_spend_de_min

In [ ]:
# Filtering for MCCs with a total spend over x amount (ignoring low value MCCs)
total_spend_de_min = total_spend_de_min[total_spend_de_min['Total Spend'] > 50000]
total_spend_de_min

In [ ]:
total_spend_de_min.to_csv('total_de_min_trade.csv')

In [ ]:
pfig2 = px.line(
        df_De_Min,
        x="time_period_value",
        y="Total Spend",
        color="destination_country")
pfig2

In [ ]:
#Looking at all transactions by UK cardholders, online, on all MCCs related to goods from Rest of ASIAPAC
UK_De_Min_2 = """SELECT time_period_value, spend, transactions, mcc, cardholder_location
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel`
WHERE time_period = 'Month'
and merchant_channel = 'Online'
and cardholder_origin = 'UNITED KINGDOM'
and mcc in ('ANTIQUE REPRODUCTION STORES',
'ANTIQUE SHOPS', 
        'ARTIST/CRAFT SHOPS',
        'ART DEALERS & GALLERIES',
        'AUTOMATED FUEL DISPENSERS',
        'AUTOMOTIVE PARTS STORES', 
        'AUTOMOTIVE TIRE STORES', 
        'BAKERIES', 
        'BICYCLE SHOPS/SALE/SERVICE',
        'BOAT DEALERS',
        'BOOKS/PERIODICALS/NEWSPAPERS',
        'BOOK STORES', 
        'CAMERA & PHOTO SUPPLY STORES',
        'CAMPER TRAILER DEALER',
        'CANDY/NUT/CONFECTION STORES',
        'CAR & TRUCK DEALERS/NEW/USED', 
        'CAR & TRUCK DEALERS/USED ONLY', 
        'CATALOG MERCHANT',
        'CHILDREN/INFANTS WEAR STORES',
        'CLOTHING/RENT/COSTUME/UNIFO',
        'COMBINATION CATALOG & RETAIL',
        'COMPUTERS/PERIPHERALS/SOFTWARE',
        'COSMETIC STORES', 
        'DAIRY PRODUCT STORES',
        'DEPARTMENT STORES', 
        'DIRECT SELL/DOOR-TO-DOOR',
        'DISCOUNT STORES',
        'DRAPERY & UPHOLSTERY STORES',
        'DUTY FREE STORES',
        'ELEC RAZOR STORES/SALE/SERV',
        'ELECTRONICS STORES', 
        'FABRIC STORES',
        'FAMILY CLOTHING STORES',
        'FIREPLACES & ACCESSORIES',
        'FLOOR COVERING STORES', 
        'FLORIST SUPPLIES/NURSERY STOCK',
        'FLORISTS',
        'FREEZER/MEAT LOCKERS',
        'FUEL DEALERS',
        'FURNITURE/EQUIP STORES',
        'FURRIERS AND FUR SHOPS',
        'GLASS/PAINT/WALLPAPER STORES', 
        'GIFT, CARD, NOVELTY STORES', 
        'GLASSWARE/CRYSTAL STORES',
        'GROCERY STORES/SUPERMARKETS',
        'HARDWARE STORES',
        'HOBBY, TOY & GAME STORES',
        'HOME SUPPLY WAREHOUSE STORES',
        'HOUSEHOLD APPLIANCE STORES', 
        'JEWELRY STORES', 
        'LUGGAGE/LEATHER STORES',
        'LUMBER/BUILD. SUPPLY STORES', 
        'MEN/BOYS CLOTHING/ACC STORES',
        'MENS/WOMENS CLOTHING STORES',
        'MISC APPAREL/ACCESS STORES',
        'MISC AUTO DEALERS - DEFAULT',
        'MISC FOOD STORES - DEFAULT',
        'MISC GENERAL MERCHANDISE', 
        'MISC HOME FURNISHING SPECIALTY', 
        'MISC SPECIALTY RETAIL', 
        'MOBILE HOME DEALERS', 
        'MOTOR HOME DEALERS',
        'MOTOR VEHICLE SUPPLY/NEW PARTS', 
        'MOTORCYCLE DEALERS', 
        'MUSIC STORES/PIANOS', 
        'NEWS DEALERS/NEWSSTANDS',
        'NURSURIES, LAWN/GARDEN SUPPLY',
        'OFFICE/PHOTO EQUIPMENT',
        'ONLINE MARKETPLACES',
        'OTHER DIRECT MARKETERS',
        'OUTBOUND TELEMARKETING MERCHNT',
        'PAINT, VARNISHES & SUPPLIES',
        'PET STORES/FOOD & SUPPLY',
        'PETROLEUM/PETROLEUM PRODUCTS',
        'PLUMBING/HEATING EQUIPMENT',
        'POSTAGE STAMPS',
        'PRECIOUS STONES/METALS/JEWELRY',
        'RECORD STORES',
        'RELIGIOUS GOODS STORES',
        'ROOFING/SIDING/SHEET METAL'
        'SERVICE STATIONS',
        'SHOE STORES',
        'SNOWMOBILE DEALERS',
        'SPORTING GOODS STORES',
        'SPORTS/RIDING APPAREL STORES',
        'STAMP & COIN STORES',
        'STATIONERY STORES',
        'STATIONERY/OFFICE SUPPLIES',
        'SWIMMING POOLS/SALES/SERV',
        'TENT AND AWNING SHOPS',
        'TELECOMMUNICATION EQUIPMENT',
        'UNIFORMS & COMMERCIAL CLOTHING',
        'USED MERCHANDISE STORES',
        'WIG AND TOUPEE STORES',
        'WOMENS READY TO WEAR STORES',
        'WRECKING SALVAGE YARDS',
'VIDEO AMUSEMENT GAME SUPPLY')
and destination_country = 'REST OF  ASIAPAC'
and cardholder_origin_country = 'POSTAL_DISTRICT'
GROUP BY time_period_value, mcc, spend, transactions, cardholder_location
ORDER BY time_period_value, mcc, cardholder_location
"""
df_De_Min_2 = bq.read_bq_table_sql(client, UK_De_Min_2)
p_district_sum = df_De_Min_2.groupby('time_period_value')['spend'].sum().reset_index()
p_district_sum
#df_De_Min_2


In [ ]:
#Looking at all transactions by UK cardholders, online, on all MCCs related to goods from Rest of ASIAPAC
UK_De_Min_3 = """SELECT time_period_value, spend, transactions, mcc, cardholder_location
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel`
WHERE time_period = 'Month'
and merchant_channel = 'Online'
and cardholder_origin = 'UNITED KINGDOM'
and mcc in ('ANTIQUE REPRODUCTION STORES',
'ANTIQUE SHOPS', 
        'ARTIST/CRAFT SHOPS',
        'ART DEALERS & GALLERIES',
        'AUTOMATED FUEL DISPENSERS',
        'AUTOMOTIVE PARTS STORES', 
        'AUTOMOTIVE TIRE STORES', 
        'BAKERIES', 
        'BICYCLE SHOPS/SALE/SERVICE',
        'BOAT DEALERS',
        'BOOKS/PERIODICALS/NEWSPAPERS',
        'BOOK STORES', 
        'CAMERA & PHOTO SUPPLY STORES',
        'CAMPER TRAILER DEALER',
        'CANDY/NUT/CONFECTION STORES',
        'CAR & TRUCK DEALERS/NEW/USED', 
        'CAR & TRUCK DEALERS/USED ONLY', 
        'CATALOG MERCHANT',
        'CHILDREN/INFANTS WEAR STORES',
        'CLOTHING/RENT/COSTUME/UNIFO',
        'COMBINATION CATALOG & RETAIL',
        'COMPUTERS/PERIPHERALS/SOFTWARE',
        'COSMETIC STORES', 
        'DAIRY PRODUCT STORES',
        'DEPARTMENT STORES', 
        'DIRECT SELL/DOOR-TO-DOOR',
        'DISCOUNT STORES',
        'DRAPERY & UPHOLSTERY STORES',
        'DUTY FREE STORES',
        'ELEC RAZOR STORES/SALE/SERV',
        'ELECTRONICS STORES', 
        'FABRIC STORES',
        'FAMILY CLOTHING STORES',
        'FIREPLACES & ACCESSORIES',
        'FLOOR COVERING STORES', 
        'FLORIST SUPPLIES/NURSERY STOCK',
        'FLORISTS',
        'FREEZER/MEAT LOCKERS',
        'FUEL DEALERS',
        'FURNITURE/EQUIP STORES',
        'FURRIERS AND FUR SHOPS',
        'GLASS/PAINT/WALLPAPER STORES', 
        'GIFT, CARD, NOVELTY STORES', 
        'GLASSWARE/CRYSTAL STORES',
        'GROCERY STORES/SUPERMARKETS',
        'HARDWARE STORES',
        'HOBBY, TOY & GAME STORES',
        'HOME SUPPLY WAREHOUSE STORES',
        'HOUSEHOLD APPLIANCE STORES', 
        'JEWELRY STORES', 
        'LUGGAGE/LEATHER STORES',
        'LUMBER/BUILD. SUPPLY STORES', 
        'MEN/BOYS CLOTHING/ACC STORES',
        'MENS/WOMENS CLOTHING STORES',
        'MISC APPAREL/ACCESS STORES',
        'MISC AUTO DEALERS - DEFAULT',
        'MISC FOOD STORES - DEFAULT',
        'MISC GENERAL MERCHANDISE', 
        'MISC HOME FURNISHING SPECIALTY', 
        'MISC SPECIALTY RETAIL', 
        'MOBILE HOME DEALERS', 
        'MOTOR HOME DEALERS',
        'MOTOR VEHICLE SUPPLY/NEW PARTS', 
        'MOTORCYCLE DEALERS', 
        'MUSIC STORES/PIANOS', 
        'NEWS DEALERS/NEWSSTANDS',
        'NURSURIES, LAWN/GARDEN SUPPLY',
        'OFFICE/PHOTO EQUIPMENT',
        'ONLINE MARKETPLACES',
        'OTHER DIRECT MARKETERS',
        'OUTBOUND TELEMARKETING MERCHNT',
        'PAINT, VARNISHES & SUPPLIES',
        'PET STORES/FOOD & SUPPLY',
        'PETROLEUM/PETROLEUM PRODUCTS',
        'PLUMBING/HEATING EQUIPMENT',
        'POSTAGE STAMPS',
        'PRECIOUS STONES/METALS/JEWELRY',
        'RECORD STORES',
        'RELIGIOUS GOODS STORES',
        'ROOFING/SIDING/SHEET METAL'
        'SERVICE STATIONS',
        'SHOE STORES',
        'SNOWMOBILE DEALERS',
        'SPORTING GOODS STORES',
        'SPORTS/RIDING APPAREL STORES',
        'STAMP & COIN STORES',
        'STATIONERY STORES',
        'STATIONERY/OFFICE SUPPLIES',
        'SWIMMING POOLS/SALES/SERV',
        'TENT AND AWNING SHOPS',
        'TELECOMMUNICATION EQUIPMENT',
        'UNIFORMS & COMMERCIAL CLOTHING',
        'USED MERCHANDISE STORES',
        'WIG AND TOUPEE STORES',
        'WOMENS READY TO WEAR STORES',
        'WRECKING SALVAGE YARDS',
'VIDEO AMUSEMENT GAME SUPPLY')
and destination_country = 'REST OF  ASIAPAC'
and cardholder_origin_country = 'POSTAL_SECTOR'
GROUP BY time_period_value, mcc, spend, transactions, cardholder_location
ORDER BY time_period_value, mcc, cardholder_location
"""
df_De_Min_3 = bq.read_bq_table_sql(client, UK_De_Min_3)
p_sector_sum = df_De_Min_3.groupby('time_period_value')['spend'].sum().reset_index()
p_sector_sum
#df_De_Min_3

In [ ]:
#Looking at all transactions by UK cardholders, online, on all MCCs related to goods from Rest of ASIAPAC
UK_De_Min_total = """SELECT time_period_value, spend, transactions, mcc, cardholder_location
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel`
WHERE time_period = 'Month'
and merchant_channel = 'Online'
and cardholder_origin = 'UNITED KINGDOM'
and mcc in ('ANTIQUE REPRODUCTION STORES',
'ANTIQUE SHOPS', 
        'ARTIST/CRAFT SHOPS',
        'ART DEALERS & GALLERIES',
        'AUTOMATED FUEL DISPENSERS',
        'AUTOMOTIVE PARTS STORES', 
        'AUTOMOTIVE TIRE STORES', 
        'BAKERIES', 
        'BICYCLE SHOPS/SALE/SERVICE',
        'BOAT DEALERS',
        'BOOKS/PERIODICALS/NEWSPAPERS',
        'BOOK STORES', 
        'CAMERA & PHOTO SUPPLY STORES',
        'CAMPER TRAILER DEALER',
        'CANDY/NUT/CONFECTION STORES',
        'CAR & TRUCK DEALERS/NEW/USED', 
        'CAR & TRUCK DEALERS/USED ONLY', 
        'CATALOG MERCHANT',
        'CHILDREN/INFANTS WEAR STORES',
        'CLOTHING/RENT/COSTUME/UNIFO',
        'COMBINATION CATALOG & RETAIL',
        'COMPUTERS/PERIPHERALS/SOFTWARE',
        'COSMETIC STORES', 
        'DAIRY PRODUCT STORES',
        'DEPARTMENT STORES', 
        'DIRECT SELL/DOOR-TO-DOOR',
        'DISCOUNT STORES',
        'DRAPERY & UPHOLSTERY STORES',
        'DUTY FREE STORES',
        'ELEC RAZOR STORES/SALE/SERV',
        'ELECTRONICS STORES', 
        'FABRIC STORES',
        'FAMILY CLOTHING STORES',
        'FIREPLACES & ACCESSORIES',
        'FLOOR COVERING STORES', 
        'FLORIST SUPPLIES/NURSERY STOCK',
        'FLORISTS',
        'FREEZER/MEAT LOCKERS',
        'FUEL DEALERS',
        'FURNITURE/EQUIP STORES',
        'FURRIERS AND FUR SHOPS',
        'GLASS/PAINT/WALLPAPER STORES', 
        'GIFT, CARD, NOVELTY STORES', 
        'GLASSWARE/CRYSTAL STORES',
        'GROCERY STORES/SUPERMARKETS',
        'HARDWARE STORES',
        'HOBBY, TOY & GAME STORES',
        'HOME SUPPLY WAREHOUSE STORES',
        'HOUSEHOLD APPLIANCE STORES', 
        'JEWELRY STORES', 
        'LUGGAGE/LEATHER STORES',
        'LUMBER/BUILD. SUPPLY STORES', 
        'MEN/BOYS CLOTHING/ACC STORES',
        'MENS/WOMENS CLOTHING STORES',
        'MISC APPAREL/ACCESS STORES',
        'MISC AUTO DEALERS - DEFAULT',
        'MISC FOOD STORES - DEFAULT',
        'MISC GENERAL MERCHANDISE', 
        'MISC HOME FURNISHING SPECIALTY', 
        'MISC SPECIALTY RETAIL', 
        'MOBILE HOME DEALERS', 
        'MOTOR HOME DEALERS',
        'MOTOR VEHICLE SUPPLY/NEW PARTS', 
        'MOTORCYCLE DEALERS', 
        'MUSIC STORES/PIANOS', 
        'NEWS DEALERS/NEWSSTANDS',
        'NURSURIES, LAWN/GARDEN SUPPLY',
        'OFFICE/PHOTO EQUIPMENT',
        'ONLINE MARKETPLACES',
        'OTHER DIRECT MARKETERS',
        'OUTBOUND TELEMARKETING MERCHNT',
        'PAINT, VARNISHES & SUPPLIES',
        'PET STORES/FOOD & SUPPLY',
        'PETROLEUM/PETROLEUM PRODUCTS',
        'PLUMBING/HEATING EQUIPMENT',
        'POSTAGE STAMPS',
        'PRECIOUS STONES/METALS/JEWELRY',
        'RECORD STORES',
        'RELIGIOUS GOODS STORES',
        'ROOFING/SIDING/SHEET METAL'
        'SERVICE STATIONS',
        'SHOE STORES',
        'SNOWMOBILE DEALERS',
        'SPORTING GOODS STORES',
        'SPORTS/RIDING APPAREL STORES',
        'STAMP & COIN STORES',
        'STATIONERY STORES',
        'STATIONERY/OFFICE SUPPLIES',
        'SWIMMING POOLS/SALES/SERV',
        'TENT AND AWNING SHOPS',
        'TELECOMMUNICATION EQUIPMENT',
        'UNIFORMS & COMMERCIAL CLOTHING',
        'USED MERCHANDISE STORES',
        'WIG AND TOUPEE STORES',
        'WOMENS READY TO WEAR STORES',
        'WRECKING SALVAGE YARDS',
'VIDEO AMUSEMENT GAME SUPPLY')
and destination_country = 'REST OF  ASIAPAC'
and cardholder_origin_country = 'All'
GROUP BY time_period_value, mcc, spend, transactions, cardholder_location
ORDER BY time_period_value, mcc, cardholder_location
"""
df_De_Min_total = bq.read_bq_table_sql(client, UK_De_Min_total)
total_sum = df_De_Min_total.groupby('time_period_value')['spend'].sum().reset_index()
total_sum
#df_De_Min_total

In [ ]:
checks_table = pd.DataFrame(total_sum)
checks_table = pd.merge(checks_table, p_area_sum, on='time_period_value', how='outer')
checks_table = checks_table.rename(columns={'spend_x': 'total spend', 'spend_y': 'p_area spend'})
checks_table = pd.merge(checks_table, p_district_sum, on='time_period_value', how='outer')
checks_table = pd.merge(checks_table, p_sector_sum, on='time_period_value', how='outer')
checks_table = checks_table.rename(columns={'spend_x': 'p_district spend', 'spend_y': 'p_sector spend'})
checks_table

In [ ]:
#Calculating how much spending is lost down each level of geographies
checks_table['p_area missing spend %'] = (checks_table['total spend']-checks_table['p_area spend'])/checks_table['total spend']
checks_table['p_district missing spend %'] = (checks_table['total spend']-checks_table['p_district spend'])/checks_table['total spend']
checks_table['p_sector missing spend %'] = (checks_table['total spend']-checks_table['p_sector spend'])/checks_table['total spend']
checks_table

In [ ]:
# Plot line chart using plotly express for all countries
pfig2 = px.line(
        df_De_Min,
        x="time_period_value",
        y="Estimated De Minimis Spend")
pfig2